# Training and Converting an LSTM Model from PyTorch to Tensorflow Lite for Micro


## Training in PyTorch

Let's train a hand-written character recognition model with the MNIST dataset which uses LSTM models.

Start off with importing the dataset:

In [71]:
import torch
import torchvision
from torchvision import transforms

"""
Each data point is of form [PIL.Image, class] where the class is a number 0-9 of the
respective digit. The image is a single channel image with values from 0 to 1.
"""
def mnist_data(train: bool) -> torchvision.datasets.MNIST:
    return torchvision.datasets.MNIST(
        './mnist/',
        download=True,
        train=train, 
        transform = transforms.Compose([
            transforms.ToTensor()
        ])
)

# iterator returning Tensors of type [batch (10), channels (1), height (28), width (28)]
data_loaders = {
    "train": torch.utils.data.DataLoader(
        mnist_data(True),
        batch_size=100,
        shuffle=True,
        num_workers=4
    ),
    "test": torch.utils.data.DataLoader(
        mnist_data(False),
        batch_size=100,
        shuffle=True,
        num_workers=4
    ),}

### Define Model

In [72]:
class MNIST_LSTM(torch.nn.Module):
    def __init__(self, input_size: int, hidden_size: int, num_layers: int = 1):
        super(MNIST_LSTM, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.input_size = input_size
        
        self.lstm = torch.nn.LSTM(
            input_size = input_size,
            hidden_size = hidden_size,
            num_layers = num_layers,
            batch_first = True
        )
        self.dropout = torch.nn.Dropout(0.2)
        self.fc = torch.nn.Linear(hidden_size, 10)
        
    def forward(self, x):
        # Set initial hidden and cell states 
        hidden_initial = torch.zeros(self.num_layers, x.size(0), self.hidden_size) 
        cell_states_initial = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        
        lstm_out, hidden = self.lstm(x, (hidden_initial, cell_states_initial))
        
        dropout_out = self.dropout(lstm_out)
        
        fc_out = self.fc(dropout_out[:, -1, :])
        return fc_out
       

### Traning Loop

In [73]:
# instantiate model
model = MNIST_LSTM(input_size=28, hidden_size=128)

In [74]:
loss_func = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

input_size=28

In [75]:
def train(num_epochs: int, model: MNIST_LSTM, loaders):
    for epoch in range(num_epochs):
        print(f"Epoch {epoch}/{num_epochs}")
        for i, (images, labels) in enumerate(loaders['train']):
            images = images.reshape(-1, input_size, input_size)
            
            outputs = model(images)
            loss = loss_func(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
    print("Ended epoch: ")

In [76]:
train(num_epochs=3, model=model, loaders=data_loaders)

Epoch 0/3
Epoch 1/3
Epoch 2/3
Ended epoch: 


In [77]:
# Test the model
model.eval()

with torch.no_grad():
    correct = 0
    total = 0
    
    for images, labels in data_loaders['test']:
        images = images.reshape(-1, input_size, input_size)
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total = total + labels.size(0)
        
        correct = correct + (predicted == labels).sum().item()
    
    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 97.52 %


### Model Usage

In [90]:
images, labels = batch = next(iter(data_loaders["test"]))

results = model(images.reshape(-1, input_size, input_size)).tolist()
processed_results = list(map(lambda res: res.index(max(res)), results))

display(torchvision.transforms.ToPILImage()(images[0]))
print(processed_results[0])

5


## Convert to ONNX

In [62]:
sample_batch = batch = next(iter(data_loaders["test"]))
sample_input = sample_batch[0].reshape(-1, input_size, input_size)

onnx_filename = "handwriting.onnx"

torch.onnx.export(
    model,
    sample_input,
    onnx_filename,
    input_names= ["image"],
    output_names=["number"]
)

## Test ONNX

In [93]:
import onnx

# Load the ONNX model
onnx_model = onnx.load(onnx_filename)

# Check that the model is well formed
onnx.checker.check_model(onnx_model)

In [112]:
import numpy
import onnxruntime as ort

ort_session = ort.InferenceSession(onnx_filename)

outputs = ort_session.run(
    None,
    {"image": sample_input.numpy()},
)
onnx_processed_results = [numpy.argmax(res) for res in outputs[0]]

print(onnx_processed_results[0])
display(torchvision.transforms.ToPILImage()(sample_input[0]))

7



## ONNX To TFlite

In [120]:
import torch
import io, os, shutil
import tensorflow as tf
import numpy as np
import tflite
from onnx2keras import onnx_to_keras
from torch.autograd import Variable


keras_model = onnx_to_keras(
        onnx_model,
        ["images"],
)


INFO:onnx2keras:Converter is called.
DEBUG:onnx2keras:List input shapes:
DEBUG:onnx2keras:None
DEBUG:onnx2keras:List inputs:
DEBUG:onnx2keras:Input 0 -> image.
DEBUG:onnx2keras:List outputs:
DEBUG:onnx2keras:Output 0 -> number.
DEBUG:onnx2keras:Gathering weights to dictionary.
DEBUG:onnx2keras:Found weight fc.weight with shape (10, 128).
DEBUG:onnx2keras:Found weight fc.bias with shape (10,).
DEBUG:onnx2keras:Found weight onnx::LSTM_118 with shape (1, 512, 28).
DEBUG:onnx2keras:Found weight onnx::LSTM_119 with shape (1, 512, 128).
DEBUG:onnx2keras:Found weight onnx::LSTM_120 with shape (1, 1024).
DEBUG:onnx2keras:Found weight onnx::Concat_121 with shape (1,).
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: Identity
DEBUG:onnx2keras:node_name: onnx::Concat_122
DEBUG:onnx2keras:node_params: {'change_ordering': False, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras

AttributeError: Current node is not in weights / model inputs / layers.

## Training in TensorFlow


In [ ]:
import keras
from keras.datasets import mnist

#Importing the data
(X_train, y_train),(X_test, y_test) = mnist.load_data() 

#Normalizing the data
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0


In [ ]:
X_train.shape[1:]

In [ ]:

#Initializing model
model = keras.models.Sequential()

#Adding the model layers
model.add(keras.layers.LSTM(128, input_shape=(X_train.shape[1:]), return_sequences=True))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.LSTM(128))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(10, activation='softmax'))

#Compiling the model
model.compile( loss='sparse_categorical_crossentropy', optimizer = keras.optimizers.Adam(lr=0.001, decay=1e-6), metrics=['accuracy'] )

#Fitting data to the model
history = model.fit(X_train, y_train, epochs=3, validation_data=(X_test, y_test))

In [ ]:
# Evaluating the test loss and accuracy
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))
